# How to Grow (almost) Anything 2021

## Remote Cloning Exercise

In this exercise, we will use the OT-2 robot to perform a complete synthetic biology workflow. As previously described, we will isolate the gene amilCP which encodes for a chromoprotein that was originally extracted from [Acropora Millepora](https://reefbuilders.com/2017/05/17/acropora-millepora/):

![](https://reefbuilders.com/wp-content/blogs.dir/1/files/2017/05/acropora-millepora-wild-3.jpg)

Below is the plan. You can choose to do it in a long day or during two sessions. Notice that you don't have to be engaged with the robot at all times - biology requires patience :)

Part A:

1. We will isolate the pUC19 backbone by restriction digest using the endonuclease enzyme PvuII (15-30 minutes)
2. We will generate mutated fragments of the amilCP gene using mismatched PCR primers (1.5 hours)
3. We will purify and quantify the fragments (this step will be performed by the TA)

Part B:

4. We will assemble the fragments and backbone using Gibson Assembly (15-30 minutes)
5. We will transform the plasmid into competent E.coli cells by heat shock (30 minutes)
6. We will incubate the transformed cells in the OpenTrons (60 minutes)
6. We will plate the cells on LB agar plates 
7. We will incubate the plates overnight and see what grows (the last step will be performed by the TA)

First, we load and initialize the OpenTrons library

In [1]:
# Re-run this cell will reset the protocol!

# import opentrons.execute
# protocol = opentrons.execute.get_protocol_api('2.9')

# Mockup
from opentrons_mock import opentrons_execute_get_mock_api
protocol = opentrons_execute_get_mock_api()
protocol.home()

Going home!


------
Next, we load all of the labware and modules for our workflow. Do not changes anything here

In [2]:
# Load labware

# Tips
tips_20ul = protocol.load_labware(
    'opentrons_96_tiprack_20ul', 
    '3', 
    'Opentrons 20uL Tips')


tips_300ul = protocol.load_labware(
    'opentrons_96_tiprack_300ul', 
    '6', 
    'Opentrons 20uL Tips')


# DNase/RNase-Free Water Reservoir
water_container = protocol.load_labware(
    'usascientific_12_reservoir_22ml', 
    '1', 
    'Water Source')


# Modules

temperature_module = protocol.load_module('temperature module gen2', 4)

thermocycler_module = protocol.load_module('thermocycler')


# Cold tray
cold_tray = temperature_module.load_labware('opentrons_24_aluminumblock_generic_2ml_screwcap', label='Cold Tray')

# Reaction Plate
reaction_plate = thermocycler_module.load_labware('nest_96_wellplate_100ul_pcr_full_skirt', label='Reaction Plate')

...
Loaded opentrons_96_tiprack_20ul in deck slot 3
...
Loaded opentrons_96_tiprack_300ul in deck slot 6
...
Loaded usascientific_12_reservoir_22ml in deck slot 1
...
Loaded module temperature module gen2 in deck slot 4
...
Loaded module thermocycler in deck slot 10
...
Module temperature module gen2 loaded opentrons_24_aluminumblock_generic_2ml_screwcap
...
Module thermocycler loaded nest_96_wellplate_100ul_pcr_full_skirt


-------
We need to configure our modules. We will use the temperature module as an "ice block" - it will keep all of our reagents and enzymes nice and cool at 4C until we are ready to mix them. The cell will finish executing only when it reaches the desired temperature - which would take a few minutes. There's a good chance the TA will already set the module to 4C before you start to save time.

For the Thermocycler module - we open the lid and set the lid temperature to 102C. Lid temperature is different than the block temperature. The goal of lod temperature is to prevent evaporation of the reaction as it goes through elevated temperature. Since the lid is always hotter than the reaction, we minimize any evaporation.

In [3]:
# Configure modules. This will take a few minutes if the modules are not already at 4C

# Set cold tray to 4C
temperature_module.set_temperature(4)

# Open Thermocycler lid
thermocycler_module.open_lid()

# Set *Lid* temperature to 102C
thermocycler_module.set_lid_temperature(102)

# And set the Block temperature to 4C
thermocycler_module.set_block_temperature(4)

...
Setting temperature to  4 C
...
Openning lid
...
Setting lid temperature to  102 C
...
Setting block temperature to  4 C


Now, pipettes. We will use two pipette for this experiment. 

Available ranges:
- P20: 1 µL - 20 µL
- P300: 20 µL - 300 µL

In [4]:
# Load pipettes
pipette_20ul = protocol.load_instrument(
    "p20_single_gen2", 
    "left",
    tip_racks=[tips_20ul])

pipette_300ul = protocol.load_instrument(
    "p300_single", 
    "right",
    tip_racks=[tips_300ul])

...
Loaded instrument p20_single_gen2 in mount left
...
Loaded instrument p20_single in mount right


In [5]:
# Choose starting tips, in case some of the tips are already used

pipette_20ul.starting_tip = tips_20ul.well('A1')
pipette_300ul.starting_tip = tips_300ul.well('A1')

In [6]:
# And test it by picking and disposing a tip

pipette_20ul.pick_up_tip()
pipette_20ul.drop_tip()

pipette_300ul.pick_up_tip()
pipette_300ul.drop_tip()

# Continue only if this step worked!

...
 p20_single_gen2 is picking up a tip from A1
...
 p20_single_gen2 is dropping a tip
...
 p20_single is picking up a tip from A1
...
 p20_single is dropping a tip


## 1. Isolate pUC19 backbone by Restriction Digest

Digest the pUC19 plasmid with the restriction enzyme PvuII to generate the linear blunt-ended backbone fragment.

First, prepare the following reaction by filling in the Volume column:

| Item | Location | Stock Concentration | Target Concentration | Volume (50uL total) |
| ---- | ----- | ----- | ---- | --- |
| DNase/RNase-Free Water | Water Reservoir | | | ? |
| 10x NEB Buffer CutSmart | Cold Tray A1 | 10X | 1X | ? |
| pUC19 | Cold Tray A2 | 1 mg/ml | 1 ug |  ? |
| PvuII Enzyme | Cold Tray A3 | 20,000 U/ml | 60 U | ? |

Then, choose an empty well in the 96-well plate in the thermocycler and transfer all of the reagents, to it.

Finally, incubate in the thermocycler for **37C** for **15-30 minutes**. To see the optimal reaction temperature for your enzyme, you can always visit the product page: [PvuII-HF](https://www.neb.com/products/r3151-pvuii-hf#Product%20Information)

In [7]:
# Restriction Digest

# Water
pipette_300ul.pick_up_tip()
pipette_300ul.aspirate(40, water_container['A1'])
pipette_300ul.dispense(40, reaction_plate['A1'])
pipette_300ul.blow_out()
pipette_300ul.drop_tip()

# Buffer
pipette_20ul.pick_up_tip()
pipette_20ul.aspirate(5, cold_tray['A1'])
pipette_20ul.dispense(5, reaction_plate['A1'])
pipette_20ul.mix(2, 10, reaction_plate['A1'])  # mix 2 times a volume of 10uL
pipette_20ul.blow_out()
pipette_20ul.drop_tip()

# pUC19 DNA 
pipette_20ul.pick_up_tip()
pipette_20ul.aspirate(2, cold_tray['A2'].bottom(0.5)) # 0.5mm from the bottom, instead of 1mm
pipette_20ul.dispense(2, reaction_plate['A1'])
pipette_20ul.mix(2, 10, reaction_plate['A1'])
pipette_20ul.blow_out()
pipette_20ul.drop_tip()

# PvuII Enzyme
pipette_20ul.pick_up_tip()
pipette_20ul.aspirate(3, cold_tray['A3'])
pipette_20ul.dispense(3, reaction_plate['A1'])
pipette_20ul.mix(2, 10, reaction_plate['A1'])
pipette_20ul.blow_out()
pipette_20ul.drop_tip()

...
 p20_single_gen2 is picking up a tip from B1
...
##### Deck Slot 1 - Water Source [ A1 ] ---> ( 40 uL)
...
##### Deck Slot 10 - Reaction Plate [ A1 ] <--- ( 40 uL)
...
 p20_single_gen2 blow out
...
 p20_single_gen2 is dropping a tip
...
 p20_single_gen2 is picking up a tip from C1
...
##### Deck Slot 4 - Cold Tray [ A1 ] ---> ( 5 uL)
...
##### Deck Slot 10 - Reaction Plate [ A1 ] <--- ( 5 uL)
...
##### Deck Slot 10 - Reaction Plate [ A1 ] - Mixing - 2 times, volume 10 uL
...
 p20_single_gen2 blow out
...
 p20_single_gen2 is dropping a tip
...
 p20_single_gen2 is picking up a tip from D1
...
##### Deck Slot 4 - Cold Tray [ A2 ] ---> ( 2 uL)
...
##### Deck Slot 10 - Reaction Plate [ A1 ] <--- ( 2 uL)
...
##### Deck Slot 10 - Reaction Plate [ A1 ] - Mixing - 2 times, volume 10 uL
...
 p20_single_gen2 blow out
...
 p20_single_gen2 is dropping a tip
...
 p20_single_gen2 is picking up a tip from E1
...
##### Deck Slot 4 - Cold Tray [ A3 ] ---> ( 3 uL)
...
##### Deck Slot 10 - Reaction Pl

In [8]:
# Start the reaction!
# Heat up the thermocycler for the desired temperature and desired time period

thermocycler_module.close_lid()
thermocycler_module.set_block_temperature(37, 
                                          hold_time_minutes=15, 
                                          block_max_volume=50) # block_max_volume should be the total reaction volume

# We're done with the restrioction digest! wait at 4C until the TA picks up the sample
thermocycler_module.set_block_temperature(4)

...\Closing lid
...
Setting block temperature to  37 C
For 15 minutes...
...
Setting block temperature to  4 C
